In [1]:
%%init_spark
launcher.master="yarn"
launcher.num_executors=6
launcher.executor_memory="6000m"


In [2]:
val links=sc.textFile("/hadoop-user/data/higgs.txt")
links.take(10).foreach(println)

Intitializing Scala interpreter ...

Spark Web UI available at http://bd-hm:8088/proxy/application_1573163713666_0001
SparkContext available as 'sc' (version = 2.4.4, master = yarn, app id = application_1573163713666_0001)
SparkSession available as 'spark'


1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11


links: org.apache.spark.rdd.RDD[String] = /hadoop-user/data/higgs.txt MapPartitionsRDD[1] at textFile at <console>:25


In [3]:
import org.apache.spark.graphx._
//creating the [pair rdd]
val edges= links.map{line=> val splits=line.split("\\s+"); (splits(0).toLong,splits(1).toLong)}
edges.take(10).foreach(println)
//creating graph rdd from edges
//edges.collect.foreach(println)
val graph=Graph.fromEdgeTuples(edges,null)

val calculatedindegree=graph.inDegrees

(1,2)
(1,3)
(1,4)
(1,5)
(1,6)
(1,7)
(1,8)
(1,9)
(1,10)
(1,11)


import org.apache.spark.graphx._
edges: org.apache.spark.rdd.RDD[(Long, Long)] = MapPartitionsRDD[2] at map at <console>:28
graph: org.apache.spark.graphx.Graph[Null,Int] = org.apache.spark.graphx.impl.GraphImpl@4e915fbf
calculatedindegree: org.apache.spark.graphx.VertexRDD[Int] = VertexRDDImpl[16] at RDD at VertexRDD.scala:57


In [4]:
val noofnodes=graph.numVertices

noofnodes: Long = 456626


In [5]:
val counts=calculatedindegree.map{case(vertex,indegree)=>(indegree,1)}.reduceByKey((indegree1,indegree2)=>indegree1+indegree2)


counts: org.apache.spark.rdd.RDD[(Int, Int)] = ShuffledRDD[19] at reduceByKey at <console>:29


In [ ]:
calculatedindegree.take(10).foreach(println)
println("******************")
counts.take(10).foreach(println)

In [6]:
val finalrdd=counts.map{case(indegree,probability)=>(indegree,probability.toDouble/noofnodes)}

finalrdd: org.apache.spark.rdd.RDD[(Int, Double)] = MapPartitionsRDD[20] at map at <console>:31


In [14]:
finalrdd.saveAsTextFile("output file")

org.apache.hadoop.mapred.FileAlreadyExistsException:  Output directory hdfs://bd-hm:9000/user/administrator/output file already exists